In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

In [12]:
df = pd.read_csv("../data/Training Data_Case 3.csv", index_col=0)
returns = df.pct_change().dropna()

In [16]:
# Define the number of assets in the portfolio
n_assets = len(df.columns)

# Define the minimum and maximum weight of each asset in the portfolio
min_weight = 0.0
max_weight = 1.0

# Define the target return and the desired risk level of the portfolio
target_return = 0.1
desired_risk = 0.2

# Define the risk tolerance parameter (affects the weighting of the risk parity allocation)
risk_tolerance = 1.0

# Define the objective function to minimize
def objective(weights):
    # Calculate the portfolio returns and variance
    portfolio_return = np.dot(weights, returns.mean()) 
    portfolio_covariance = np.cov(returns.T) 
    portfolio_variance = np.dot(weights.T, np.dot(portfolio_covariance, weights))

    # Calculate the skew and kurtosis of the returns distribution
    skewness = np.sum((returns.mean() - portfolio_return)**3) / (n_assets * portfolio_variance**(3/2))
    kurtosis = np.sum((returns.mean() - portfolio_return)**4) / (n_assets * portfolio_variance**2) - 3

    # Calculate the contribution of each asset to the risk parity allocation
    asset_risk_contributions = np.sqrt(np.diag(portfolio_covariance)) * weights / np.sqrt(portfolio_variance)
    asset_risk_contributions /= np.sum(asset_risk_contributions)

    # Calculate the objective function value
    obj = (portfolio_return - target_return)**2 + risk_tolerance * np.sum((asset_risk_contributions - 1/n_assets)**2)
    obj += skewness**2 + kurtosis**2

    return obj

# Define the initial guess for the weights of each asset in the portfolio
init_guess = np.ones(n_assets) / n_assets

# Define the weight constraints for the optimization
weight_constraints = [{'type': 'ineq', 'fun': lambda x: x[i] - min_weight} for i in range(n_assets)]
weight_constraints += [{'type': 'ineq', 'fun': lambda x: -x[i] + max_weight} for i in range(n_assets)]
weight_constraints += [{'type': 'eq', 'fun': lambda x: np.sum(x) - 1.0}]

# Perform the optimization
result = minimize(objective, init_guess, constraints=weight_constraints)

# Calculate the portfolio returns, variance, and Sharpe Ratio
portfolio_return = np.dot(result.x, returns.mean()) 
portfolio_covariance = np.cov(returns.T) 
portfolio_variance = np.dot(result.x.T, np.dot(portfolio_covariance, result.x))
sharpe_ratio = (252 ** 0.5) * (portfolio_return) / np.sqrt(portfolio_variance)

# Print the optimized portfolio weights and Sharpe Ratio
print("Optimized Portfolio Weights: ", result.x)
print("Sharpe Ratio: ", sharpe_ratio)


Optimized Portfolio Weights:  [0.10210583 0.1302953  0.14138723 0.09678279 0.09904126 0.06973882
 0.0899162  0.06236273 0.12945536 0.07891449]
Sharpe Ratio:  1.05524736920883
